In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
# from sklearn.feature_extraction.text import TfidVectorizer

In [15]:
df_movies=pd.read_csv("tmdb_5000_movies.csv")
df_credits=pd.read_csv("tmdb_5000_credits.csv")

In [18]:
df_movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4773,27000,"[{""id"": 35, ""name"": ""Comedy""}]",http://www.miramax.com/movie/clerks/,2292,"[{""id"": 1361, ""name"": ""salesclerk""}, {""id"": 30...",en,Clerks,Convenience and video store clerks Dante and R...,19.748658,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1994-09-13,3151130,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Just because they serve you doesn't mean they ...,Clerks,7.4,755
4781,22000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",https://www.facebook.com/DrySpellMovie,255266,"[{""id"": 13043, ""name"": ""dating""}, {""id"": 15160...",en,Dry Spell,Sasha tries to get her soon-to-be ex husband K...,0.048948,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-02-14,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Getting divorced does funny things to a girl,Dry Spell,6.0,1
4791,13,"[{""id"": 27, ""name"": ""Horror""}]",http://tincanmanthemovie.com/,157185,"[{""id"": 14903, ""name"": ""home invas

In [17]:
df_movies.dropna(inplace=bool(1),axis=0)

In [19]:
df_movies['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4773    Convenience and video store clerks Dante and R...
4781    Sasha tries to get her soon-to-be ex husband K...
4791    Recently dumped by his girlfirend for another ...
4796    Friends/fledgling entrepreneurs invent a devic...
4801    When ambitious New York attorney Sam is sent t...
Name: overview, Length: 1493, dtype: object

In [20]:
tfidf=CountVectorizer(stop_words="english")
df_movies['overview']=df_movies['overview'].fillna('')
tfidf_matrix=tfidf.fit_transform(df_movies['overview'])

In [21]:
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [22]:
indices=pd.Series(df_movies.index,index=df_movies['original_title']).drop_duplicates()
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
Clerks                                      4773
Dry Spell                                   4781
Tin Can Man                                 4791
Primer                                      4796
Shanghai Calling                            4801
Length: 1493, dtype: int64

In [52]:
indices['Avatar']

0

In [53]:
def get_recommendation(title, cosine_sim=cosine_sim):
    idx=indices[title]
    sim_scores=enumerate(cosine_sim[idx])
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores=sim_scores[1:11]
#     for i in sim_scores:
#         print(i)
    sim_index=[i[0] for i in sim_scores]
    print(sim_index)
    print(df_movies['original_title'].iloc[sim_index],df_movies['homepage'].iloc[sim_index])


In [54]:
get_recommendation('Avatar')

[1243, 447, 579, 117, 1091, 30, 39, 61, 118, 146]
3604                                            Apollo 18
847                                              Semi-Pro
1213                          Aliens vs Predator: Requiem
150                                       Men in Black II
2967                           E.T. the Extra-Terrestrial
36                        Transformers: Age of Extinction
45                                            World War Z
74                                       Edge of Tomorrow
151                                               Beowulf
199     Pirates of the Caribbean: The Curse of the Bla...
Name: original_title, dtype: object 3604                            http://apollo18movie.net/
847            http://newline.com/properties/semipro.html
1213                                http://www.avp-r.com/
150     http://www.sonypictures.com/homevideo/meninbla...
2967                                 http://www.et20.com/
36                       http://www.transfor

In [51]:
get_recommendation('The Matrix')

[638, 1360, 1140, 1407, 57, 63, 198, 301, 545, 618]
1389                     Old Dogs
4174    El Crimen del Padre Amaro
3149                       Volver
4388                     Hardflip
69                           Hugo
77                     Inside Out
284                Little Fockers
510               Children of Men
1137            One for the Money
1337                     Twilight
Name: original_title, dtype: object 1389                        http://disney.go.com/olddogs/
4174          http://padreamaro.ad2.com/site/home_eng.htm
3149                     http://www.volver-lapelicula.com
4388                            http://hardflipmovie.com/
69                              http://www.hugomovie.com/
77                    http://movies.disney.com/inside-out
284                         http://www.littlefockers.net/
510     http://www.universalstudiosentertainment.com/c...
1137        http://www.oneforthemoneyfilm.com/index2.html
1337                      http://www.twilightthemo